In [3]:
from ROOT import *
import numpy as np
from prettytable import PrettyTable
import sys
import uncertainties as unc # propagate uncertainties
from uncertainties import unumpy as unp # array operations for type ufloat

# this script draw quark efficiency, gluon rejection , scale factor for given quark working point e.g. 0.6, 
# pdf is not included in the plot (can be added in line 684 and 686) because nan exist in old pdf sample 
doreweight = "Quark"   #decide if we want to do the reweighting process

wpoint = 0.5

var = "ntrk"  #change the var name according to the inputvar you want to read

inputvar = var  #by setting it as bdt (or ntrk,width,c1..), it will read the corresponding histogram, but remember to change the TLine range according to X-axis of different variable, one can check it by browsing the histograms in root file.

bins = np.array([500.,600.,800.,1000.,1200.,1500.,2000.])
bin = np.array([500,600,800,1000,1200,1500,2000])
if var == "ntrk":
    doreweight = "Quark"
    var_text = "N_{trk}"
if var == "bdt":
    doreweight = 0
    var_text = "BDT score"

def error_check(hist):
    for i in range(1,hist.GetNbinsX()+1):
        if hist.GetBinContent(i)>1:
            hist.SetBinContent(i,0)
    return(hist)

# convert histogram into uncertainty array
def unc_array(hist):
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = hist.GetBinError(j)
    result = unp.uarray(value,error)
    return(result)

# convert uncertainty array into histogram
def set_hist(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))
        hist.SetBinContent(i,unp.nominal_values(unc[i-1]))
        
def ReadingDijetMCindex(file0,pt,var,name,index):
    histo_dic = {}
    higher_quark = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var)


    higher_quark.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+var+name)
    lower_quark.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+var+name)
    higher_gluon.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+var+name)
    

    higher_quark_2 = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark_2 = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon_2 = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon_2 = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark_2 = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark_2 = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark_2 = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark_2 = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var)

    higher_quark_2.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+var+name)
    lower_quark_2.SetName(pt+"_SubJet_Central_Quark_"+str(index)+var+name)
    higher_gluon_2.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon_2.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark_2.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark_2.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark_2.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark_2.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+var+name)

    higher_quark.Add(higher_quark_2)
    higher_gluon.Add(higher_gluon_2)
    higher_b_quark.Add(higher_b_quark_2)
    higher_c_quark.Add(higher_c_quark_2)

    lower_quark.Add(lower_quark_2)
    lower_gluon.Add(lower_gluon_2)
    lower_b_quark.Add(lower_b_quark_2)
    lower_c_quark.Add(lower_c_quark_2)
 
            
    higher_b_quark.Add(higher_c_quark)
    lower_b_quark.Add(lower_c_quark)

    higher_quark_err = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_err = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_err = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_err = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_err = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_err = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_err = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_err = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err")



    
    higher_quark_err.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_err.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_err.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_err.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_err.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_err.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_err.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_err.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_2_err = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_2_err = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_2_err = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_2_err = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_2_err = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_2_err = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_2_err = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_2_err = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err")
    
    higher_quark_2_err.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_2_err.SetName(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_2_err.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_2_err.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_2_err.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_2_err.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_2_err.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_2_err.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_err.Add(higher_quark_2_err)
    higher_gluon_err.Add(higher_gluon_2_err)
    higher_b_quark_err.Add(higher_b_quark_2_err)
    higher_c_quark_err.Add(higher_c_quark_2_err)

    lower_quark_err.Add(lower_quark_2_err)
    lower_gluon_err.Add(lower_gluon_2_err)
    lower_b_quark_err.Add(lower_b_quark_2_err)
    lower_c_quark_err.Add(lower_c_quark_2_err)
 
            
    higher_b_quark_err.Add(higher_c_quark_err)
    lower_b_quark_err.Add(lower_c_quark_err)
    
    for i in range(1,higher_quark.GetNbinsX()+1):
        higher_quark.SetBinError(i,np.sqrt(higher_quark_err.GetBinContent(i)))
        higher_gluon.SetBinError(i,np.sqrt(higher_gluon_err.GetBinContent(i)))
        higher_b_quark.SetBinError(i,np.sqrt(higher_b_quark_err.GetBinContent(i)))
        lower_quark.SetBinError(i,np.sqrt(lower_quark_err.GetBinContent(i)))
        lower_gluon.SetBinError(i,np.sqrt(lower_gluon_err.GetBinContent(i)))
        lower_b_quark.SetBinError(i,np.sqrt(lower_b_quark_err.GetBinContent(i)))

    
    histo_dic["hq"] =  higher_quark
    histo_dic["hg"] =  higher_gluon
    histo_dic["ho"] =  higher_b_quark
    
    histo_dic["hmc"] =  higher_quark.Clone()
    histo_dic["hmc"].SetName(pt+"higher_all_jets"+var+"_"+name)
    histo_dic["hmc"].Add(higher_gluon)
    histo_dic["hmc"].Add(higher_b_quark)
    
    histo_dic["lq"] =  lower_quark
    histo_dic["lg"] =  lower_gluon
    histo_dic["lo"] =  lower_b_quark
    
    histo_dic["lmc"] =  lower_quark.Clone()
    histo_dic["lmc"].SetName(pt+"lower_all_jets"+var+"_"+name)
    histo_dic["lmc"].Add(lower_gluon)
    histo_dic["lmc"].Add(lower_b_quark)
    
    return(histo_dic)


def ReadingDijetMC(file0,pt,var,name):
    histo_dic = {}
    higher_quark = file0.Get(pt+"_LeadingJet_Forward_Quark_"+var)
    lower_quark = file0.Get(pt+"_LeadingJet_Central_Quark_"+var)
    higher_gluon = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+var)
    lower_gluon = file0.Get(pt+"_LeadingJet_Central_Gluon_"+var)
    higher_b_quark = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+var)
    lower_b_quark = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+var)
    higher_c_quark = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+var)
    lower_c_quark = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+var)

    
    higher_quark.SetName(pt+"_LeadingJet_Forward_Quark_"+var+name)
    lower_quark.SetName(pt+"_LeadingJet_Central_Quark_"+var+name)
    higher_gluon.SetName(pt+"_LeadingJet_Forward_Gluon_"+var+name)
    lower_gluon.SetName(pt+"_LeadingJet_Central_Gluon_"+var+name)
    higher_b_quark.SetName(pt+"_LeadingJet_Forward_B_Quark_"+var+name)
    lower_b_quark.SetName(pt+"_LeadingJet_Central_B_Quark_"+var+name)
    higher_c_quark.SetName(pt+"_LeadingJet_Forward_C_Quark_"+var+name)
    lower_c_quark.SetName(pt+"_LeadingJet_Central_C_Quark_"+var+name)
    

    higher_quark_2 = file0.Get(pt+"_SubJet_Forward_Quark_"+var)
    lower_quark_2 = file0.Get(pt+"_SubJet_Central_Quark_"+var)
    higher_gluon_2 = file0.Get(pt+"_SubJet_Forward_Gluon_"+var)
    lower_gluon_2 = file0.Get(pt+"_SubJet_Central_Gluon_"+var)
    higher_b_quark_2 = file0.Get(pt+"_SubJet_Forward_B_Quark_"+var)
    lower_b_quark_2 = file0.Get(pt+"_SubJet_Central_B_Quark_"+var)
    higher_c_quark_2 = file0.Get(pt+"_SubJet_Forward_C_Quark_"+var)
    lower_c_quark_2 = file0.Get(pt+"_SubJet_Central_C_Quark_"+var)


    higher_quark_2.SetName(pt+"_SubJet_Forward_Quark_"+var+name)
    lower_quark_2.SetName(pt+"_SubJet_Central_Quark_"+var+name)
    higher_gluon_2.SetName(pt+"_SubJet_Forward_Gluon_"+var+name)
    lower_gluon_2.SetName(pt+"_SubJet_Central_Gluon_"+var+name)
    higher_b_quark_2.SetName(pt+"_SubJet_Forward_B_Quark_"+var+name)
    lower_b_quark_2.SetName(pt+"_SubJet_Central_B_Quark_"+var+name)
    higher_c_quark_2.SetName(pt+"_SubJet_Forward_C_Quark_"+var+name)
    lower_c_quark_2.SetName(pt+"_SubJet_Central_C_Quark_"+var+name)

    higher_quark.Add(higher_quark_2)
    higher_gluon.Add(higher_gluon_2)
    higher_b_quark.Add(higher_b_quark_2)
    higher_c_quark.Add(higher_c_quark_2)

    lower_quark.Add(lower_quark_2)
    lower_gluon.Add(lower_gluon_2)
    lower_b_quark.Add(lower_b_quark_2)
    lower_c_quark.Add(lower_c_quark_2)
 
            
    higher_b_quark.Add(higher_c_quark)
    lower_b_quark.Add(lower_c_quark)

    higher_quark_err = file0.Get(pt+"_LeadingJet_Forward_Quark_"+var+"_err")
    lower_quark_err = file0.Get(pt+"_LeadingJet_Central_Quark_"+var+"_err")
    higher_gluon_err = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+var+"_err")
    lower_gluon_err = file0.Get(pt+"_LeadingJet_Central_Gluon_"+var+"_err")
    higher_b_quark_err = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+var+"_err")
    lower_b_quark_err = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+var+"_err")
    higher_c_quark_err = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+var+"_err")
    lower_c_quark_err = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+var+"_err")

    
    higher_quark_err.SetName(pt+"_LeadingJet_Forward_Quark_"+var+"_err"+name)
    lower_quark_err.SetName(pt+"_LeadingJet_Central_Quark_"+var+"_err"+name)
    higher_gluon_err.SetName(pt+"_LeadingJet_Forward_Gluon_"+var+"_err"+name)
    lower_gluon_err.SetName(pt+"_LeadingJet_Central_Gluon_"+var+"_err"+name)
    higher_b_quark_err.SetName(pt+"_LeadingJet_Forward_B_Quark_"+var+"_err"+name)
    lower_b_quark_err.SetName(pt+"_LeadingJet_Central_B_Quark_"+var+"_err"+name)
    higher_c_quark_err.SetName(pt+"_LeadingJet_Forward_C_Quark_"+var+"_err"+name)
    lower_c_quark_err.SetName(pt+"_LeadingJet_Central_C_Quark_"+var+"_err"+name)
    
    higher_quark_2_err = file0.Get(pt+"_SubJet_Forward_Quark_"+var+"_err")
    lower_quark_2_err = file0.Get(pt+"_SubJet_Central_Quark_"+var+"_err")
    higher_gluon_2_err = file0.Get(pt+"_SubJet_Forward_Gluon_"+var+"_err")
    lower_gluon_2_err = file0.Get(pt+"_SubJet_Central_Gluon_"+var+"_err")
    higher_b_quark_2_err = file0.Get(pt+"_SubJet_Forward_B_Quark_"+var+"_err")
    lower_b_quark_2_err = file0.Get(pt+"_SubJet_Central_B_Quark_"+var+"_err")
    higher_c_quark_2_err = file0.Get(pt+"_SubJet_Forward_C_Quark_"+var+"_err")
    lower_c_quark_2_err = file0.Get(pt+"_SubJet_Central_C_Quark_"+var+"_err")


    higher_quark_2_err.SetName(pt+"_SubJet_Forward_Quark_"+var+"_err"+name)
    lower_quark_2_err.SetName(pt+"_SubJet_Central_Quark_"+var+"_err"+name)
    higher_gluon_2_err.SetName(pt+"_SubJet_Forward_Gluon_"+var+"_err"+name)
    lower_gluon_2_err.SetName(pt+"_SubJet_Central_Gluon_"+var+"_err"+name)
    higher_b_quark_2_err.SetName(pt+"_SubJet_Forward_B_Quark_"+var+"_err"+name)
    lower_b_quark_2_err.SetName(pt+"_SubJet_Central_B_Quark_"+var+"_err"+name)
    higher_c_quark_2_err.SetName(pt+"_SubJet_Forward_C_Quark_"+var+"_err"+name)
    lower_c_quark_2_err.SetName(pt+"_SubJet_Central_C_Quark_"+var+"_err"+name)

    higher_quark_err.Add(higher_quark_2_err)
    higher_gluon_err.Add(higher_gluon_2_err)
    higher_b_quark_err.Add(higher_b_quark_2_err)
    higher_c_quark_err.Add(higher_c_quark_2_err)

    lower_quark_err.Add(lower_quark_2_err)
    lower_gluon_err.Add(lower_gluon_2_err)
    lower_b_quark_err.Add(lower_b_quark_2_err)
    lower_c_quark_err.Add(lower_c_quark_2_err)
 
            
    higher_b_quark_err.Add(higher_c_quark_err)
    lower_b_quark_err.Add(lower_c_quark_err)
    
    for i in range(1,higher_quark.GetNbinsX()+1):
        higher_quark.SetBinError(i,np.sqrt(higher_quark_err.GetBinContent(i)))
        higher_gluon.SetBinError(i,np.sqrt(higher_gluon_err.GetBinContent(i)))
        higher_b_quark.SetBinError(i,np.sqrt(higher_b_quark_err.GetBinContent(i)))
        lower_quark.SetBinError(i,np.sqrt(lower_quark_err.GetBinContent(i)))
        lower_gluon.SetBinError(i,np.sqrt(lower_gluon_err.GetBinContent(i)))
        lower_b_quark.SetBinError(i,np.sqrt(lower_b_quark_err.GetBinContent(i)))

    
    histo_dic["hq"] =  higher_quark
    histo_dic["hg"] =  higher_gluon
    histo_dic["ho"] =  higher_b_quark
    histo_dic["hmc"] =  higher_quark.Clone()

    histo_dic["hmc"].SetName(pt+"higher_all_jets"+var+"_"+name)
    histo_dic["hmc"].Add(higher_gluon)
    histo_dic["hmc"].Add(higher_b_quark)
    
    histo_dic["lq"] =  lower_quark
    histo_dic["lg"] =  lower_gluon
    histo_dic["lo"] =  lower_b_quark
    histo_dic["lmc"] =  lower_quark.Clone()
    
    histo_dic["lmc"].SetName(pt+"lower_all_jets"+var+"_"+name)
    histo_dic["lmc"].Add(lower_gluon)
    histo_dic["lmc"].Add(lower_b_quark)
    
    return(histo_dic)

def ReadingDijetData(file0,pt,var,name):
    histo_dic = {}
    higher_data = file0.Get(pt+"_LeadingJet_Forward_Data_"+var)
    lower_data = file0.Get(pt+"_LeadingJet_Central_Data_"+var)

    higher_data.SetName(pt+"_LeadingJet_Forward_Data_"+var+name)
    lower_data.SetName(pt+"_LeadingJet_Central_Data_"+var+name)
    
    higher_data_2 = file0.Get(pt+"_SubJet_Forward_Data_"+var)
    lower_data_2 = file0.Get(pt+"_SubJet_Central_Data_"+var)
    
    higher_data_2.SetName(pt+"_SubJet_Forward_Data_"+var+name)
    lower_data_2.SetName(pt+"_SubJet_Central_Data_"+var+name)
    
    higher_data.Add(higher_data_2)
    lower_data.Add(lower_data_2)
    

    higher_data_err = file0.Get(pt+"_LeadingJet_Forward_Data_"+var+"_err")
    lower_data_err = file0.Get(pt+"_LeadingJet_Central_Data_"+var+"_err")

    higher_data_err.SetName(pt+"_LeadingJet_Forward_Data_"+var+"_err"+name)
    lower_data_err.SetName(pt+"_LeadingJet_Central_Data_"+var+"_err"+name)

    higher_data_2_err = file0.Get(pt+"_SubJet_Forward_Data_"+var+"_err")
    lower_data_2_err = file0.Get(pt+"_SubJet_Central_Data_"+var+"_err")

    higher_data_2_err.SetName(pt+"_SubJet_Forward_Data_"+var+"_err"+name)
    lower_data_2_err.SetName(pt+"_SubJet_Central_Data_"+var+"_err"+name)

    higher_data_err.Add(higher_data_2_err)
    lower_data_err.Add(lower_data_2_err)
    
    for i in range(1,higher_data.GetNbinsX()+1):
        higher_data.SetBinError(i,np.sqrt(higher_data_err.GetBinContent(i)))
        lower_data.SetBinError(i,np.sqrt(lower_data_err.GetBinContent(i)))
        
    histo_dic["hd"] =  higher_data
    histo_dic["ld"] =  lower_data

    return(histo_dic)        

def myText(x,y,text, color = 1):
    l = TLatex()
    l.SetTextSize(0.025)
    l.SetNDC()
    l.SetTextColor(color)
    l.DrawLatex(x,y,text)
    pass
# input the mc sample(must be unnormalized)

#inclusive fraction calculation
def fraction(lower_quark,lower_gluon,higher_quark,higher_gluon):
    ToT_Fq2 = 0.
    ToT_Fg2 = 0.
    ToT_Cq2 =0.
    ToT_Cg2 = 0.

    for j in range(1,lower_quark.GetNbinsX()+1):
        ToT_Fq2+=higher_quark.GetBinContent(j)
        ToT_Cq2+=lower_quark.GetBinContent(j)
        ToT_Fg2+=higher_gluon.GetBinContent(j)
        ToT_Cg2+=lower_gluon.GetBinContent(j)

    # calculate the fraction of each sample 
    if ((ToT_Fg2+ToT_Fq2) != 0):
        fg=ToT_Fg2/(ToT_Fg2+ToT_Fq2)
        cg=ToT_Cg2/(ToT_Cq2+ToT_Cg2)

    fq=1.-fg
    cq=1.-cg
    return(fg,cg,fq,cq)

## do matrix on mc for given normalized distribution
def mc_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,fg,cg,fq,cq,higher_mc_input,lower_mc_input):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower_mc = lower_mc_input.Clone()
        higher_mc = higher_mc_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_quark)


            
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_gluon)

        
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower_mc.Integral() != 0):
                lower_mc.Scale(1./lower_mc.Integral())
        if(higher_mc.Integral() != 0):
                higher_mc.Scale(1./higher_mc.Integral())

        higher = higher_mc.Clone()
        lower = lower_mc.Clone()          


        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)

#return the abs systematic uncertainty
def mc_error(mc_quark_1,mc_gluon_1,mc_quark_2,mc_gluon_2): 
    error_q = np.zeros(mc_quark_1.GetNbinsX())
    error_g = np.zeros(mc_gluon_1.GetNbinsX())

    for j in range(1,mc_gluon_1.GetNbinsX()+1):
                q = abs(mc_quark_1.GetBinContent(j) - mc_quark_2.GetBinContent(j))
                g = abs(mc_gluon_1.GetBinContent(j) - mc_gluon_2.GetBinContent(j))
                error_q[j-1] = q
                error_g[j-1] = g

    return error_q,error_g

# do matrix on data for given normalized distribution
def data_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,higher_input,lower_input,fg,cg,fq,cq):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower = lower_input.Clone()
        higher = higher_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_quark)    
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_gluon)                                
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower.Integral() != 0):
            lower.Scale(1./lower.Integral()) 
        if(higher.Integral() != 0):
            higher.Scale(1./higher.Integral())         
        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()
        #if min == 500:
        #    for j in range(1,61):
        #        print("testflag3:",j,higher.GetBinContent(j),lower.GetBinContent(j))

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)
    
    
# Get the bin(cut) of given working point for the nominal result and the Quark efficiency , Gluon rejection , scale factor.
def wp_sf_nominal(wpoint,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])
    
    mc_bin = np.abs(quark_mc_cumsum -  wpoint*quark_mc.Integral(1,quark_mc.GetNbinsX()+1)).argmin()
    data_bin = np.abs(quark_data_cumsum -  wpoint*quark_data.Integral(1,quark_data.GetNbinsX()+1)).argmin()
    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(mc_bin,data_bin,sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

# Get the Quark efficiency , Gluon rejection , scale factor for systematics.
def wp_sf(mc_bin,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])

    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

#def bootstrap_result(data):
#    w = 58.45/39.91 # prescale factor 
#    n1 = data.Clone()
#    n2 = data.Clone()
#    result = data.Clone()
#    for j in range(1,higher_quark.GetNbinsX()+1):
#            n2.SetBinContent(j,int((data.GetBinContent(j)-data.GetBinError(j)**2)/(w-w**2)))#(events num for prescaled data)
#            n1.SetBinContent(j,int((data.GetBinContent(j)-w*n2.GetBinContent(j))))
#            result.SetBinContent(j,w*np.random.poisson(n2.GetBinContent(j))+np.random.poisson(n1.GetBinContent(j)))
#    return(result)  

def bootstrap_result(data):
    n1 = data.Clone()
    result = data.Clone()
    for j in range(1,higher_quark.GetNbinsX()+1):
            result.SetBinContent(j,np.random.poisson(n1.GetBinContent(j)))
    return(result)  

gStyle.SetOptStat(0)
c = TCanvas("","",500,500)
#gPad.SetTickx()
#gPad.SetTicky()


qeff = []
grej = []
qsf_array = []
gsf_array = []
unc_array_q = []
unc_array_g = []

                
ntrackall3 = TFile("/eos/user/r/rqian/dijet-mono-result/data/dijet_data.root")
ntrackall4 = TFile("/eos/user/r/rqian/dijet-mono-result/pythia/dijet_pythia_mono.root")



for i in range(0,6):


            min = bin[i]
            max = bin[i+1]
            print(min)

            nominal_jets =  ReadingDijetMC(ntrackall4,str(min),var,"mc")
            
            higher_quark = nominal_jets["hq"]
            lower_quark = nominal_jets["lq"]
            higher_gluon = nominal_jets["hg"]
            lower_gluon = nominal_jets["lg"]
            higher_other = nominal_jets["ho"]
            lower_other = nominal_jets["lo"]        
            higher_mc = nominal_jets["hmc"]
            lower_mc = nominal_jets["lmc"]    

            data_jets = ReadingDijetData(ntrackall3,str(min),var,"data")

            higher_data_TH1I = data_jets["hd"]
            higher_data = higher_quark.Clone(higher_data_TH1I.GetName()+"TH1F")
            for j in range(1,higher_data.GetNbinsX()+1):
                    higher_data.SetBinContent(j,float(higher_data_TH1I.GetBinContent(j)))
                    higher_data.SetBinError(j,float(higher_data_TH1I.GetBinError(j)))             
            
            lower_data_TH1I = data_jets["ld"]
            lower_data = lower_quark.Clone(lower_data_TH1I.GetName()+"TH1F")
            for j in range(1,higher_data.GetNbinsX()+1):
                    lower_data.SetBinContent(j,float(lower_data_TH1I.GetBinContent(j)))
                    lower_data.SetBinError(j,float(lower_data_TH1I.GetBinError(j))) 
                    
            
            higher_quark_unc = unc_array(higher_quark)
            higher_gluon_unc = unc_array(higher_gluon)
            lower_quark_unc = unc_array(lower_quark)
            lower_gluon_unc = unc_array(lower_gluon)
            higher_mc_unc = unc_array(higher_mc)
            lower_mc_unc = unc_array(lower_mc)
            
            ToT_Fq2_unc = higher_quark_unc.sum()
            ToT_Fg2_unc = higher_gluon_unc.sum()
    
            ToT_Cq2_unc = lower_quark_unc.sum()
            ToT_Cg2_unc = lower_gluon_unc.sum()
    
            # calculate the fraction of forward(higher) / central(lower) quark or gluon jet
            fg_unc=ToT_Fg2_unc/(ToT_Fg2_unc+ToT_Fq2_unc)
            cg_unc=ToT_Cg2_unc/(ToT_Cq2_unc+ToT_Cg2_unc)
            fq_unc=1.-fg_unc
            cq_unc=1.-cg_unc
    
            factor_quark_unc = lower_quark_unc
            factor_gluon_unc = lower_gluon_unc
    
            higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
            higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
            lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
            lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()

            higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
            lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()        
        
            if (doreweight=="Quark"):
                for j in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(j) > 0 and lower_gluon.GetBinContent(j) > 0):
                                factor_gluon_unc[j-1] = higher_gluon_unc[j-1]/lower_gluon_unc[j-1]
                                factor_quark_unc[j-1] = higher_quark_unc[j-1]/lower_quark_unc[j-1]
                        else:
                                factor_gluon_unc[j-1] = unc.ufloat(1, 0)
                                factor_quark_unc[j-1] = unc.ufloat(1, 0)
                lower_quark_unc=lower_quark_unc*factor_quark_unc
                lower_gluon_unc=lower_gluon_unc*factor_quark_unc
                lower_mc_unc=lower_mc_unc*factor_quark_unc

            if (doreweight=="Gluon"):
                for j in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(j) > 0 and lower_gluon.GetBinContent(j) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon_unc[j-1] = higher_gluon_unc[j-1]/lower_gluon_unc[j-1]
                                factor_quark_unc[j-1] = higher_quark_unc[j-1]/lower_quark_unc[j-1]
                        else:
                                factor_gluon_unc[j-1] = unc.ufloat(1, 0)
                                factor_quark_unc[j-1] = unc.ufloat(1, 0)
                lower_quark_unc=lower_quark_unc*factor_gluon_unc
                lower_gluon_unc=lower_gluon_unc*factor_gluon_unc
                lower_mc_unc=lower_mc_unc*factor_gluon_unc
            
            higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
            higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
            lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
            lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()

            higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
            lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()   
            
            higher = higher_mc.Clone()
            lower = lower_mc.Clone()
    
            higher_unc = higher_mc_unc
            lower_unc  = lower_mc_unc
    
            
            # matrix method here
            F_unc = higher_unc
            C_unc = lower_unc
            Q_unc = -(C_unc*fg_unc-F_unc*cg_unc)/(cg_unc*fq_unc-fg_unc*cq_unc)
            G_unc = (C_unc*fq_unc-F_unc*cq_unc)/(cg_unc*fq_unc-fg_unc*cq_unc)
            

            
            #compute the uncertainty and plots


            higher_data_nominal = higher_data.Clone()
            lower_data_nominal = lower_data.Clone()
            
            
            #do matrix method on data (nominal fraction)
            # first normalize it
            if (higher_data_nominal.Integral() != 0):
                higher_data_nominal.Scale(1/higher_data_nominal.Integral())
            if (lower_data_nominal.Integral() != 0):
                lower_data_nominal.Scale(1/lower_data_nominal.Integral())
            # do matrix method to extract the distribution of sherpa first
            fg,cg,fq,cq = fraction(lower_quark,lower_gluon,higher_quark,higher_gluon)
            
            
            # normalize the nominal mc
            if (lower_quark.Integral() != 0):
                lower_quark.Scale(1./lower_quark.Integral())
            if(lower_gluon.Integral() != 0):
                lower_gluon.Scale(1./lower_gluon.Integral())
            if(higher_quark.Integral() != 0):
                higher_quark.Scale(1./higher_quark.Integral())
            if(higher_gluon.Integral() != 0):
                higher_gluon.Scale(1./higher_gluon.Integral())            
            if(lower_mc.Integral() != 0):
                lower_mc.Scale(1./lower_mc.Integral())
            if(higher_mc.Integral() != 0):
                higher_mc.Scale(1./higher_mc.Integral())      
            
            
            #Get nominal distribution , cut bin , efficiency/rejection and scale factors
            nominal_extract_Q,nominal_extract_G = mc_matrixmethod(lower_quark,lower_gluon,higher_quark,higher_gluon,fg,cg,fq,cq,higher_mc,lower_mc)
            extracted_data_nominal_Q,extracted_data_nominal_G = data_matrixmethod(lower_quark,lower_gluon,higher_quark,higher_gluon,higher_data_nominal,lower_data_nominal,fg,cg,fq,cq)
            mc_bin,data_bin,sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data=wp_sf_nominal(wpoint,nominal_extract_Q,nominal_extract_G,extracted_data_nominal_Q,extracted_data_nominal_G)
            
            

            higher_data_strap = higher_data.Clone()     #Set aside for statistical uncertainty
            lower_data_strap = lower_data.Clone()


            #statistical
            # do bootstrap(not normalized yet)
            # create lists to store bootstrapped values list of arrays of nstraps values
            nstraps = 5000
            SF_Qvals = []
            SF_Gvals = []


            higher_quark_bootstrap = higher_quark.Clone()
            higher_gluon_bootstrap = higher_gluon.Clone()
            lower_quark_bootstrap = lower_quark.Clone()
            lower_gluon_bootstrap = lower_gluon.Clone()
            higher_data_bootstrap = higher_data.Clone()
            lower_data_bootstrap = lower_data.Clone() 
            
            for k in range(nstraps):

                    lower_data_bootstrap = lower_data.Clone()
                    higher_data_bootstrap = higher_data.Clone()
                    #print(higher_data_bootstrap.GetBinContent(3))
                    lower_data_bootstrap = bootstrap_result(lower_data_bootstrap)
                    higher_data_bootstrap = bootstrap_result(higher_data_bootstrap)

                    if(lower_data_bootstrap.Integral() != 0):
                            lower_data_bootstrap.Scale(1./lower_data_bootstrap.Integral())
                    if(higher_data_bootstrap.Integral() != 0):
                            higher_data_bootstrap.Scale(1./higher_data_bootstrap.Integral())
                    
                    if (doreweight=="Quark"):
                            for j in range(1,higher_quark_bootstrap.GetNbinsX()+1):
                                    if (lower_quark_bootstrap.GetBinContent(j) > 0 and lower_gluon_bootstrap.GetBinContent(j) > 0):
                                            #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                            factor_gluon = higher_gluon_bootstrap.GetBinContent(j)/lower_gluon_bootstrap.GetBinContent(j)
                                            factor_quark = higher_quark_bootstrap.GetBinContent(j)/lower_quark_bootstrap.GetBinContent(j)
                                            lower_data_bootstrap.SetBinContent(j,lower_data_bootstrap.GetBinContent(j)*factor_quark)
                                            pass
                                    pass
                            pass
                        
                    if (doreweight=="Gluon"):
                            for j in range(1,higher_quark_bootstrap.GetNbinsX()+1):
                                    if (lower_quark_bootstrap.GetBinContent(j) > 0 and lower_gluon_bootstrap.GetBinContent(j) > 0):
                                            #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                            factor_gluon = higher_gluon_bootstrap.GetBinContent(j)/lower_gluon_bootstrap.GetBinContent(j)
                                            factor_quark = higher_quark_bootstrap.GetBinContent(j)/lower_quark_bootstrap.GetBinContent(j)
                                            lower_data_bootstrap.SetBinContent(j,lower_data_bootstrap.GetBinContent(j)*factor_gluon)
                                            pass
                                    pass
                            pass
                        

                    if(lower_data_bootstrap.Integral() != 0):
                            lower_data_bootstrap.Scale(1./lower_data_bootstrap.Integral())
                    if(higher_data_bootstrap.Integral() != 0):
                            higher_data_bootstrap.Scale(1./higher_data_bootstrap.Integral())
                            
                    forward_data_strap = higher_data_bootstrap.Clone(higher_data_bootstrap.GetName()+"f"+str(k))
                    central_data_strap = lower_data_bootstrap.Clone(higher_data_bootstrap.GetName()+"c"+str(k))
                            
                    if forward_data_strap.Integral() != 0:
                        forward_data_strap.Scale(1/forward_data_strap.Integral())
                    if central_data_strap.Integral() != 0:
                        central_data_strap.Scale(1/central_data_strap.Integral())                            
                            
                    bootstrap_extracted_Q = central_data_strap.Clone()
                    bootstrap_extracted_G = central_data_strap.Clone()

                    # get extracted data Q/G with sherpa sample
                    for j in range(1,higher_quark.GetNbinsX()+1):
                            F_data = forward_data_strap.GetBinContent(j)
                            C_data = central_data_strap.GetBinContent(j)
                            Q_data = -(C_data*fg-F_data*cg)/(cg*fq-fg*cq)
                            G_data = (C_data*fq-F_data*cq)/(cg*fq-fg*cq)
                            bootstrap_extracted_Q.SetBinContent(j,Q_data)
                            bootstrap_extracted_G.SetBinContent(j,G_data)
                    
                    sf_q_bootstrap,sf_g_bootstrap,q_eff_mc_bootstrap,g_rej_mc_bootstrap,q_eff_data_bootstrap,g_rej_data_bootstrap = wp_sf(mc_bin,nominal_extract_Q,nominal_extract_G,bootstrap_extracted_Q,bootstrap_extracted_G)

                    
                    SF_Qvals.append(sf_q_bootstrap)
                    SF_Gvals.append(sf_g_bootstrap)


                    Bootstrap_quark = TH1F("","",50,np.min(SF_Qvals),np.max(SF_Qvals))
                    Bootstrap_gluon = TH1F("","",50,np.min(SF_Gvals),np.max(SF_Gvals))
                    
            for j in range(5000):
                Bootstrap_quark.Fill(SF_Qvals[j])
                Bootstrap_gluon.Fill(SF_Gvals[j])
            c = TCanvas("","",500,500)
            
            Bootstrap_quark.GetXaxis().SetTitle("Scale Factor")
            Bootstrap_quark.GetYaxis().SetTitle("Frequency")
            Bootstrap_quark.Draw("hist")
            
            
            
            myText(0.16,0.84,"#it{#bf{#scale[1.2]{#bf{ATLAS} Internal}}}")
            myText(0.16,0.80,"#bf{#scale[1]{#sqrt{s} = 13 TeV, 139 fb^{-1}}}")
            myText(0.16,0.76,"#bf{#scale[1]{Quark " +var_text+ " Scale Factor}}")
            myText(0.16,0.72,"#bf{#scale[1]{"+str(int(wpoint*100))+"% Quark Efficiency}}")
            myText(0.16,0.68,"#bf{#scale[1]{p_{T}: "+str(min)+"-"+str(max)+"GeV}}")
            
            
            
            c.Print(str(min)+"-"+str(wpoint)+"-"+var+"-quark.pdf")
            
            c1 = TCanvas("","",500,500)
            
            Bootstrap_gluon.GetXaxis().SetTitle("Scale Factor")
            Bootstrap_gluon.GetYaxis().SetTitle("Frequency")
            Bootstrap_gluon.Draw("hist")
            
            
            
            myText(0.16,0.84,"#it{#bf{#scale[1.2]{#bf{ATLAS} Internal}}}")
            myText(0.16,0.80,"#bf{#scale[1]{#sqrt{s} = 13 TeV, 139 fb^{-1}}}")
            myText(0.16,0.76,"#bf{#scale[1]{Gluon " +var_text+ " Scale Factor}}")
            myText(0.16,0.72,"#bf{#scale[1]{"+str(int(wpoint*100))+"% Quark Efficiency}}")
            myText(0.16,0.68,"#bf{#scale[1]{p_{T}: "+str(min)+"-"+str(max)+"GeV}}")
            
            
            
            c1.Print(str(min)+"-"+str(wpoint)+"-"+var+"-gluon.pdf")

500
600
800
1000
1200
1500


Info in <TCanvas::Print>: pdf file 500-0.5-ntrk-quark.pdf has been created
Info in <TCanvas::Print>: pdf file 500-0.5-ntrk-gluon.pdf has been created
Info in <TCanvas::Print>: pdf file 600-0.5-ntrk-quark.pdf has been created
Info in <TCanvas::Print>: pdf file 600-0.5-ntrk-gluon.pdf has been created
Info in <TCanvas::Print>: pdf file 800-0.5-ntrk-quark.pdf has been created
Info in <TCanvas::Print>: pdf file 800-0.5-ntrk-gluon.pdf has been created
Info in <TCanvas::Print>: pdf file 1000-0.5-ntrk-quark.pdf has been created
Info in <TCanvas::Print>: pdf file 1000-0.5-ntrk-gluon.pdf has been created
Info in <TCanvas::Print>: pdf file 1200-0.5-ntrk-quark.pdf has been created
Info in <TCanvas::Print>: pdf file 1200-0.5-ntrk-gluon.pdf has been created
Info in <TCanvas::Print>: pdf file 1500-0.5-ntrk-quark.pdf has been created
Info in <TCanvas::Print>: pdf file 1500-0.5-ntrk-gluon.pdf has been created


In [2]:
nominal_extract_Q[10]

2043146170412.9927